In [1]:
import tqdm
import torch
from torch.utils.data import DataLoader
import tensorflow as tf

from robomimic.utils.rlds_utils import droid_dataset_transform, robomimic_transform, robomimic_dg_transform, TorchRLDSDataset

from octo.data.dataset import make_dataset_from_rlds, make_interleaved_dataset, make_single_dataset
from octo.data.utils.data_utils import combine_dataset_statistics
from octo.utils.spec import ModuleSpec

tf.config.set_visible_devices([], "GPU")
from octo.utils.spec import ModuleSpec
import deligrasp_dataset_transform

NameError: name 'Dict' is not defined

In [ ]:
import importlib
import robomimic
import numpy as np
importlib.reload(robomimic.utils.rlds_utils)

<module 'robomimic.utils.rlds_utils' from 'c:\\workspace\\deligrasp_policy_learning\\robomimic\\utils\\rlds_utils.py'>

In [38]:
import tensorflow_datasets as tfds
builder = tfds.builder_from_directory(builder_dir=f"{DATA_PATH}/{DATASET_NAMES}/1.0.0")
ds = tfds.load(f"{DATASET_NAMES}", data_dir=f"{DATA_PATH}", split="train")
trajectory = next(iter(next(iter(ds))['steps']))
gripper_force = trajectory["action"][7]

# for i in ds:
#     for j in i['steps']:
#         print(j['observation']['state'][-3])

In [22]:
importlib.reload(robomimic.utils.rlds_utils)
DATA_PATH = "C:/Users/willi/tensorflow_datasets"    # UPDATE WITH PATH TO RLDS DATASETS
DATASET_NAMES = "droid_100"
EXP_LOG_PATH = "C:/workspace/deligrasp_policy_learning/logs" # UPDATE WITH PATH TO DESIRED LOGGING DIRECTORY
sample_weights = [1]

# import tensorflow_datasets as tfds
# builder = tfds.builder_from_directory(f"{DATA_PATH}/1.0.0")
# builder.info.features

BASE_DATASET_KWARGS = {
    "name": "droid_100",
    "data_dir": DATA_PATH,
    "image_obs_keys": {"primary": "exterior_image_1_left", "secondary": "exterior_image_2_left"},
    "state_obs_keys": ["cartesian_position", "gripper_position"],
    "language_key": "language_instruction",
    "action_proprio_normalization_type": "bounds",
    "action_normalization_mask": [True] * 9 + [False],      # don't normalize final (gripper) dimension
    "standardize_fn": ModuleSpec.create("robomimic.utils.rlds_utils:droid_dataset_transform"),
    # "standardize_fn": droid_dataset_transform,
    # "train": True,
}

stats = make_dataset_from_rlds(**BASE_DATASET_KWARGS, train=True)
# combined_dataset_statistics = combine_dataset_statistics(
#     [make_dataset_from_rlds(**dataset_kwargs, train=True)[1] for dataset_kwargs in dataset_kwargs_list]
# )


In [24]:
stats

(<DLataset element_spec={'observation': {'image_primary': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'image_secondary': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'proprio': TensorSpec(shape=(None, 16), dtype=tf.float32, name=None), 'timestep': TensorSpec(shape=(None,), dtype=tf.int32, name=None)}, 'task': {'language_instruction': TensorSpec(shape=(None,), dtype=tf.string, name=None)}, 'action': TensorSpec(shape=(None, 11), dtype=tf.float32, name=None), 'dataset_name': TensorSpec(shape=(None,), dtype=tf.string, name=None)}>,
 {'action': {'mean': array([ 2.59525876e-07, -8.98353755e-06,  6.26606061e-06,  1.00000000e+00,
          -3.94223480e-06,  1.17622549e-05,  3.95866209e-06,  1.00000000e+00,
           1.83136490e-06, -1.90163068e-02,  1.45091570e-03]),
   'std': array([4.99562520e-05, 5.33768907e-05, 1.40456337e-04, 6.31753039e-07,
          1.29881650e-04, 1.99309536e-04, 1.29631822e-04, 5.01916190e-07,
          2.10918428e-04, 9.86317545e-02, 1.46769136e

In [65]:
dataset = make_single_dataset(
    BASE_DATASET_KWARGS,
    train=True,
    traj_transform_kwargs=dict(
        window_size=2,
        action_horizon=15,
        subsample_length=20,
        skip_unlabeled=True,            # skip all trajectories without language annotation
    ),
    frame_transform_kwargs=dict(
        image_augment_kwargs=dict(
        ),
        resize_size=dict(
            primary=[128, 128],
            secondary=[128, 128],
        ),
        num_parallel_calls=200,
    )
)

In [66]:
dataset = dataset.map(robomimic_dg_transform, num_parallel_calls=48)


In [106]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import transforms
import torch

class test(torch.utils.data.IterableDataset):
    """Thin wrapper around RLDS dataset for use with PyTorch dataloaders."""

    def __init__(
        self,
        rlds_dataset,
        train=True,
    ):
        self._rlds_dataset = rlds_dataset
        self._is_train = train
        self.transform = transforms.Compose([transforms.ToTensor()])

    def __iter__(self):
        for sample in self._rlds_dataset.as_numpy_iterator():
            print(type(sample))
            yield sample

    def __len__(self):
        lengths = np.array(
            [
                stats["num_transitions"]
                for stats in self._rlds_dataset.dataset_statistics
            ]
        )
        if hasattr(self._rlds_dataset, "sample_weights"):
            lengths *= np.array(self._rlds_dataset.sample_weights)
        total_len = lengths.sum()
        if self._is_train:
            return int(0.95 * total_len)
        else:
            return int(0.05 * total_len)


In [107]:
pytorch_dataset = test(dataset)
train_loader = DataLoader(
    pytorch_dataset,
    batch_size=1,
    num_workers=0,  # important to keep this to 0 so PyTorch does not mess with the parallelism
)

dli = iter(train_loader)
batch = next(dli)

<class 'dict'>


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found object

In [100]:
for i, sample in tqdm.tqdm(enumerate(train_loader)):
    print("enumerating")
    print(sample)
    print(i)
    if i == 5000:
        break

0it [00:03, ?it/s]


TypeError: pic should be PIL Image or ndarray. Got <class 'dict'>

In [ ]:
# find trajectory length for each trajectory in dataset
traj_lengths = []
for traj in dataset:
    print(traj['obs']['camera/image/varied_camera_1_left_image'].shape)
    
i = 0
s = None
for sample in pytorch_dataset:
    print(sample)
    s = sample
    i += 1
    if i > 1:
        break

# ok = s['obs'].keys()
# ak = s['actions']

# for k in ok:
#     print(f"type of {k}: {type(s['obs'][k])}")

In [ ]:
from robomimic.scripts.config_gen.helper import *
import random
import json
import numpy as np
from collections import OrderedDict

EXP_LOG_PATH = "C:/workspace/deligrasp_policy_learning/logs" # UPDATE WITH PATH TO DESIRED LOGGING DIRECTORY


In [ ]:
from robomimic.scripts.config_gen.helper import *
import random
import json
import numpy as np
from collections import OrderedDict

#############################################################################
# *************** Replace with your paths/config information ****************

# Note: Assumes naming of dataset in "datasets" for the full DROID dataset is
# droid

DATA_PATH = "C:/Users/willi/tensorflow_datasets/deligrasp_dataset_scaled"    # UPDATE WITH PATH TO RLDS DATASETS
EXP_LOG_PATH = "C:/workspace/deligrasp_policy_learning/logs" # UPDATE WITH PATH TO DESIRED LOGGING DIRECTORY
EXP_NAMES = OrderedDict(
    [
        # Note: you can add co-training dataset here appending
        # a new dataset to "datasets" and adjusting "sample_weights"
        # accordingly
        ("droid", {"datasets": ["droid"],
                   "sample_weights": [1]})                                    
    ])

#############################################################################

def make_generator_helper(args):
    algo_name_short = "diffusion_policy"

    generator = get_generator(
        algo_name="diffusion_policy",
        config_file=os.path.join(base_path, 'robomimic/exps/templates/diffusion_policy_test.json'),
        args=args,
        exp_log_path=EXP_LOG_PATH,
        algo_name_short=algo_name_short,
        pt=True,
    )
    if args.ckpt_mode is None:
        args.ckpt_mode = "off"

    generator.add_param(
        key="train.data_format",
        name="",
        group=-1,
        values=[
            "droid_rlds"
        ],
    )

    generator.add_param(
        key="train.num_epochs",
        name="",
        group=-1,
        values=[100000],
    )

    generator.add_param(
        key="train.data_path",
        name="",
        group=-1,
        values=[DATA_PATH],
    )

    generator.add_param(
        key="train.shuffle_buffer_size",
        name="",
        group=-1,
        values=[500000],
    )

    generator.add_param(
        key="train.batch_size",
        name="bz",
        group=1212111,
        values=[128],
        hidename=False,
    )

    generator.add_param(
        key="train.subsample_length",
        name="subsample_length",
        group=7070707,
        values=[
            100
        ],
        hidename=True,
    )

    generator.add_param(
        key="train.num_parallel_calls",
        name="num_parallel_calls",
        group=404040404,
        values=[
            200
        ],
        hidename=True,
    )

    generator.add_param(
        key="train.traj_transform_threads",
        name="traj_transform_threads",
        group=303030303,
        values=[
            48
        ],
        hidename=True,
    )

    generator.add_param(
        key="train.traj_read_threads",
        name="traj_read_threads",
        group=908090809,
        values=[
            48
        ],
        hidename=True,
    )

    generator.add_param(
        key="algo.noise_samples",
        name="noise_samples",
        group=1010101,
        values=[8],
        value_names=["8"]
    )

    # use ddim by default
    generator.add_param(
        key="algo.ddim.enabled",
        name="ddim",
        group=1001,
        values=[
            True,
            # False,
        ],
        hidename=True,
    )
    generator.add_param(
        key="algo.ddpm.enabled",
        name="ddpm",
        group=1001,
        values=[
            False,
            # True,
        ],
        hidename=True,
    )

    if args.env == "deligrasp":
        generator.add_param(
            key="train.data",
            name="ds",
            group=2,
            values=[
                [
                    {"path": "~/datasets/square/ph/square_ph_abs_tmp.hdf5"}, # replace with your own path
                ],
            ],
            value_names=[
                "square",
            ],
        )

        # update env config to use absolute action control
        generator.add_param(
            key="experiment.env_meta_update_dict",
            name="",
            group=-1,
            values=[
                {"env_kwargs": {"controller_configs": {"control_delta": False}}}
            ],
        )
        
        generator.add_param(
            key="train.action_keys",
            name="ac_keys",
            group=-1,
            values=[
                [
                    "action_dict/abs_pos",
                    "action_dict/abs_rot_6d",
                    "action_dict/gripper",
                    # "actions",
                ],
            ],
            value_names=[
                "abs",
            ],
        )
    
    elif args.env == "droid":
        generator.add_param(
            key="train.sample_weights",
            name="sample_weights",
            group=24988,
            values=[
                EXP_NAMES[k]["sample_weights"] for k in EXP_NAMES.keys()
            ],
        )
        generator.add_param(
            key="train.dataset_names",
            name="dataset_names",
            group=24988,
            values=[
                EXP_NAMES[k]["datasets"] for k in EXP_NAMES.keys()
            ],
            value_names=list(EXP_NAMES.keys())
        )
        generator.add_param(
            key="train.action_keys",
            name="ac_keys",
            group=-1,
            values=[
                [
                    "action/abs_pos",
                    "action/abs_rot_6d",
                    "action/gripper_position",
                ],
            ],
            value_names=[
                "abs",
            ],
            hidename=True,
        )
        generator.add_param(
            key="train.action_shapes",
            name="ac_shapes",
            group=-1,
            values=[
                [
                    (1, 3),
                    (1, 6),
                    (1, 1),
                ],
            ],
            value_names=[
                "ac_shapes",
            ],
            hidename=True,
        )
        generator.add_param(
            key="observation.image_dim",
            name="",
            group=-1,
            values=[
                [128, 128],
            ],
            hidename=True,
        )
        generator.add_param(
            key="observation.modalities.obs.rgb",
            name="cams",
            group=130,
            values=[
                # ["camera/image/hand_camera_left_image"],
                # ["camera/image/hand_camera_left_image", "camera/image/hand_camera_right_image"],
                ["camera/image/varied_camera_1_left_image", "camera/image/varied_camera_2_left_image"],
                # [
                    # "camera/image/hand_camera_left_image", "camera/image/hand_camera_right_image",
                #     "camera/image/varied_camera_1_left_image", "camera/image/varied_camera_1_right_image",
                #     "camera/image/varied_camera_2_left_image", "camera/image/varied_camera_2_right_image",
                # ],
            ],
            value_names=[
                # "wrist",
                # "wrist-stereo",
                "2cams",
                # "3cams-stereo",
            ]
        )
        generator.add_param(
            key="observation.encoder.rgb.obs_randomizer_class",
            name="obsrand",
            group=130,
            values=[
                # "ColorRandomizer", # jitter only
                ["ColorRandomizer", "CropRandomizer"], # jitter, followed by crop
            ],
            hidename=True,
        )
        generator.add_param(
            key="observation.encoder.rgb.obs_randomizer_kwargs",
            name="obsrandargs",
            group=130,
            values=[
                # {}, # jitter only
                [{}, {"crop_height": 116, "crop_width": 116, "num_crops": 1, "pos_enc": False}], # jitter, followed by crop
            ],
            hidename=True,
        )

        ### CONDITIONING
        generator.add_param(
            key="train.goal_mode",
            name="goal_mode",
            group=24986,
            values = [
                # "geom",
                None, # Change this to "geom" to do goal conditioning

            ]
        )
        generator.add_param(
            key="train.truncated_geom_factor",
            name="truncated_geom_factor",
            group=5555,
            values = [
                0.3,
                # 0.5
            ]
        )
        generator.add_param(
            key="observation.modalities.obs.low_dim",
            name="ldkeys",
            group=24986,
            values=[
                ["robot_state/cartesian_position", "robot_state/gripper_position"],
            ],
            value_names=[
                "proprio-lang",
            ],
            hidename=False,
        )
        generator.add_param(
            key="observation.encoder.rgb.core_kwargs.backbone_kwargs.use_cam",
            name="",
            group=2498,
            values=[
                False,
                # True,
            ],
            hidename=True,
        )
        generator.add_param(
            key="observation.encoder.rgb.core_kwargs.backbone_kwargs.pretrained",
            name="",
            group=2498,
            values=[
                # False,
                True,
            ],
            hidename=True,
        )
        generator.add_param(
            key="observation.encoder.rgb.core_class",
            name="visenc",
            group=-1,
            values=["VisualCore"],
        )
        generator.add_param(
            key="observation.encoder.rgb.core_kwargs.backbone_class",
            name="",
            group=-1,
            values=["ResNet50Conv"],
            hidename=True,
        )
        generator.add_param(
            key="observation.encoder.rgb.core_kwargs.feature_dimension",
            name="visdim",
            group=1234,
            values=[
                512,
                # None,
                # None
            ],
            hidename=True,
        )
        generator.add_param(
            key="observation.encoder.rgb.core_kwargs.flatten",
            name="flatten",
            group=1234,
            values=[
                True,
                # False,
                # False
            ],
            hidename=True,
        )
        generator.add_param(
            key="observation.encoder.rgb.fuser",
            name="fuser",
            group=1234,
            values=[
                None,
                # "transformer",
                # "perceiver"
            ],
            hidename=False,
        )
        generator.add_param(
            key="observation.encoder.rgb.core_kwargs.backbone_kwargs.downsample",
            name="",
            group=1234,
            values=[
                False,
            ],
            hidename=False,
        )

    else:
        raise ValueError
    
    generator.add_param(
        key="train.output_dir",
        name="",
        group=-1,
        values=[
            "{exp_log_path}/{env}/{mod}/{algo_name_short}".format(
                exp_log_path=EXP_LOG_PATH,
                env=args.env,
                mod=args.mod, 
                algo_name_short=algo_name_short,
            )
        ],
    )

    return generator

if __name__ == "__main__":
    parser = get_argparser()

    args = parser.parse_args()
    make_generator(args, make_generator_helper)